In [ ]:
%load_ext autoreload
%autoreload 2
from model import run_exp
import utils as utl
from dataAPI import runAPIcall

# computational set up
Choose One:
## Flywire

In [ ]:
# # Set Up Environment for Flywire Data and Setting Path for Custon Neuron Names
# path_comp = './Flywire_Data/2023_03_23_completeness_630_final.csv'
# path_con = './Flywire_Data/2023_03_23_connectivity_630_final.parquet'
# path_res = './results/example/'
# path_name = './Flywire_Data/flywire_ids_630.xlsx'

# # sheet names of the xls file to include
# sheets_pair = [ # pair of neurons (left+right per row)
#     'stop',
#     'walk',
#     'walk_outputs',
#     ]

# sheets_single = [ # single neurons (one per row)
#     # 'sugar', 
#     # 'ovidn', 
#     # 'bitter', 
#     ]

# name2flyid = utl.create_name_dict(path_name, path_comp, sheets_pair, sheets_single)


## MANC

In [ ]:
# Set Up Environment for MANC Data and Setting Path for Custon Neuron Names
path_comp = './MANC_Data/2023_06_06_completeness_1.0_final.csv'
path_con = './MANC_Data/2023_06_06_connectivity_1.0_final.parquet'
path_res = './results/example/'
path_name = './MANC_Data/MANC_ids_1.0.xlsx'

config = {
    'path_comp'         : path_comp, 
    'path_con'          : path_con, 
    'path_res'          : path_res,     # store results here
    'n_proc'            : 1,            # number of CPUs to use, -1 uses all available CPU
}

# sheet names of the xls file to include
sheets_pair = [ # pair of neurons (left+right per row)
    'stop1-type', 
    'walk', 
    'walk_outputs', 
    'stop-1 outputs', 
    'walk_inputs', 
    ]

sheets_single = [ # single neurons (one per row)
    'Pleural_remotor_abductor', 
    'Sternal_posterior_rotator', 
    'Tergopleural_Pleural promotor', 
    'Sternal_anterior_rotator', 
    'Sternal_adductor', 
    'Sternotrochanter', 
    'Tergotr', 
    'Tr_extensor', 
    'Tr_flexor', 
    'Acc_tr_flexor', 
    'Fe_reductor', 
    'Ti_extensor', 
    'Acc_Ti_flexor', 
    'Ti_flexor', 
    'LTM', 
    'Ta_depressor ', 
    'Ta_levator'
    ]


name2flyid = utl.create_name_dict(path_name, path_comp, sheets_pair, sheets_single)

In [ ]:
# lists of neuron groups
l_hind = ['hind_l','hind_r']
l_mid = ['mid_l','mid_r']
l_front = ['front_l','front_r',]

# run experiments

In [ ]:
# Hind, Mid, Rear Activation
instructions = [ 
    (0, 'stim', l_hind),
    (0, 'stim', l_mid), 
    (0, 'stim', l_front), 
    (1, 'end', [])
    ]

run_exp(exp_name='all-stim', exp_inst=instructions, name2flyid=name2flyid, **config, force_overwrite=True)

In [ ]:
# Hind Activation, Mid and Front Silencing
instructions = [ 
    (0, 'stim', l_hind),
    (1, 'end', [])
    ]

run_exp(exp_name='hind-stim', exp_inst=instructions, name2flyid=name2flyid, **config, force_overwrite=True)

In [ ]:
#Mid Activation, Hind and Front Silencing
instructions = [ 
    (0, 'stim', l_mid), 
    (1, 'end', [])
    ]

run_exp(exp_name='mid-stim', exp_inst=instructions, name2flyid=name2flyid, **config, force_overwrite=True)

In [ ]:
#Front Activation, Mid and Hind Silencing
instructions = [ 
    (0, 'stim', l_front), 
    (1, 'end', [])
    ]

run_exp(exp_name='front-stim', exp_inst=instructions, name2flyid=name2flyid, **config, force_overwrite=True)

# Process results

In [ ]:
# load experiments
outputs = [
    './results/example/all-stim.parquet',
    './results/example/hind-stim.parquet',
    './results/example/mid-stim.parquet',
    './results/example/front-stim.parquet'
]

df_spkt = utl.load_exps(outputs)
df_rate, df_std = utl.get_rate(df_spkt, duration=1)

df_rate = utl.rename_index(df_rate, name2flyid)
df_std = utl.rename_index(df_std, name2flyid)

utl.save_xls(df_rate, './results/example/rate.xlsx')
utl.save_xls(df_std, './results/example/rate_std.xlsx')

In [ ]:
# raster plots
neu = l_hind + l_mid + l_front
utl.plot_raster(df_spkt, neu, name2flyid=name2flyid, xlims=(0, 1))

In [ ]:
# firing rates
utl.plot_rate(df_spkt, neu, xlims=(0, 1), name2flyid=name2flyid)

In [ ]:
# select top 20 neurons
top20 = df_rate.sort_values(by='front-stim', ascending=False).head(20)
top20

In [ ]:
# plot heatmap
utl.plot_rate_heatmap(df_spkt, top20.index, xlims=(0, 1), name2flyid=name2flyid, do_zscore=False)

# Graph representation

In [ ]:
# load connectome into graph
G = utl.get_full_graph(path_comp, path_con)
print(len(G.nodes))

In [ ]:
# select subgraph based on simulation results
output = './results/example/front-stim.parquet'
utl.write_graph(G, output, name2flyid=name2flyid)